# Pregunta 3  Tarea 2
# Primera Parte: Preparar información necesaria para la Inversión



## GF5013 - Metodos Inversos Aplicados a la Geofisica
## Primavera 2019
### Prof. Francisco Hernán Ortega Culaciati 
### ortega.francisco@uchile.cl
### Departamento de Geofísica - FCFM
### Universidad de Chile
### Octubre de 2019

Nombre Alumno: XXXX YYYY

In [ ]:
import sys, os
pathTarea2 = '../'
pathPaquetesPython = pathTarea2 + '/PaquetesPython'
TomoDataFilename = pathTarea2 + '/datosTomo/datosTomografia.txt'

sys.path.append(os.path.abspath(pathPaquetesPython))



In [ ]:
import matplotlib
# descomentar esto es para hacer los plots interactivos en el notebook
%matplotlib notebook 
# descomentar esto es para hacer los plots afuera del notebook
#%matplotlib tk  
import matplotlib.pyplot as plt
import numpy as NP

from Tarea2.TomoData import readTravelTimeData as readTTdata
from Tarea2 import TomographyModel as Tmodel
import Tarea2
import GF5013

## Definir la malla del problema

In [ ]:
Xmin = 0
Xmax = 100
Nx = 21

Zmin = -200
Zmax = 0
Nz = 43

mesh = Tmodel.define_Tomography_RectangularMesh(Xmin = Xmin, Xmax = Xmax, Nx = Nx, Zmin = Zmin, Zmax = Zmax, Nz = Nz)


## Formar matriz de diseño del problema de tomografía lineal

In [ ]:
# LEER DATOS TOMOGRAFIA
TTdata = readTTdata(TomoDataFilename)
# Coordenadas Fuentes
xF = TTdata['xF']
zF = TTdata['zF']
# Coordenadas Receptores
xR = TTdata['xR']
zR = TTdata['zR']
# Tiempo de Viaje
TV = TTdata['TV']

import timeit
startTime = timeit.default_timer()

# formar matriz G y vector de datos G (ESTO SE DEMORA ENTRE 1 y 2 MINUTOS!!!)
G, d, rays = Tmodel.formar_G(xF, zF, xR, zR, TV, mesh, fractional_precision=0.01)

stopTime = timeit.default_timer()
totalTime = stopTime - startTime
mins, secs = divmod(totalTime, 60)
hours, mins = divmod(mins, 60)
print("Total running time: {:02d}:{:02d}:{:02d}.\n".format(int(hours), int(mins), int(secs)))

In [ ]:
# Graficar la norma de las columnas de la matrix G (una medida de la densidad de rayos
# que pasa por cada casillero)
Ndata, Npar = G.shape
F = NP.diag( G.T.dot(G) ).reshape((Npar,1) ) # se grafica la norma de las funciones de green en cada casillero
F = NP.sqrt(F)  
colorlim = [0, NP.max(F)]
fig = plt.figure(1)
fig.clear()
ax = plt.subplot(111)
fig, ax = Tarea2.Mesh.plotRectMeshField(F, mesh, fig, ax , colorlim = colorlim, \
                                 title = "Norma de las columnas de G",\
                                 colorbar_label = 'Norma de G[:,i] ', xlabel = 'X [m]',\
                                 ylabel = 'Y [m]', colormap = 'nipy_spectral')
fig.show()

In [ ]:
# generar datos sinteticos # NO MODIFIQUE LA LINEA SIGUIENTE!!!
d_syn, m_syn =  Tmodel.calcCheckerboardSynthetic(mesh, G, shift=[16.156, 16.156], scale=33.333)
# para la matrix de covarianza de los datos Cd se asume que las observaciones son iid con desviación estandar igual A
A = 1
# matriz de covarianza de los datos
Cd = # COMPLETAR
# errores de la predicción del modelo (se asume un modelo perfecto)
Cp = # COMPLETAR
# Matriz de covarianza del ajuste
Cx = # COMPLETAR

# dibujar la malla con el modelo sintético
F = 1/m_syn # se grafica la velocidad ( no la lentitud)
fig = plt.figure(2)
fig.clear()
ax = plt.subplot(111)
fig, ax = Tarea2.Mesh.plotRectMeshField(F, mesh, fig, ax , colorlim = None, \
                                 title = "Modelo Sintético",\
                                 colorbar_label = 'Velocity ', xlabel = 'X [m]',\
                                 ylabel = 'Y [m]', colormap = 'gray')
fig.show()

## Definir la regularización y la matriz de pesos del ajuste

In [ ]:
# REGULARIZACION: se usará Tikhonov de orden 2, para ello se define el laplaciano H
H, Wh, ho = Tarea2.Mesh.TikhonovReg.order_2(mesh)

# necesitamos definir la matriz de pesos del ajuste Wx
inv_Cx = # COMPLETAR
Wx = # COMPLETAR

## Guardar los G, d, y la malla para ser usados en la inversión en otro notebook


In [ ]:
import pickle
# nombre del archivo "pickle" donde guardar los datos de interés
PickleFilename = "Tomo_pre_inversion_data.pickle"

# variable a guardar es un diccionario con G, d y la malla
data2save = {}
data2save['G'] = G
data2save['d_syn'] = d_syn
data2save['Cx'] = Cx
data2save['m_syn'] = m_syn
data2save['TTdata'] = TTdata
data2save['mesh'] = mesh
data2save['H'] = H
data2save['Wh'] = Wh
data2save['ho'] = ho
data2save['inv_Cx'] = inv_Cx
data2save['Wx'] = Wx

# save the dictionray in the file.
PickleFile = open(PickleFilename, 'wb') # must be a binary file w: writable, b: binary
pickle.dump(data2save, PickleFile, pickle.HIGHEST_PROTOCOL)
PickleFile.close()

